In [13]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/hateful-memes

/content/drive/MyDrive/hateful-memes


In [136]:
# Integrate Fairface Labels

# load files
train_df = pd.read_csv('train.csv')
face_features = pd.read_csv('cyw_features/race_gender_age_all.csv')

# make sure all original text ends with ending punctuation: . ! ?
for index, row in train_df.iterrows():
    if row['text'][-1].isalpha():
        train_df.loc[index, 'text'] += "."

# cancatenate race features to text
for index, row in face_features.iterrows():
    id = row['face_name_align'][15:20]
    face_text = " " + row['race'] + " " + row['gender'] + ", " + row['age'] + " years old."
    idx = train_df[train_df['id'] == int(id)].index
    train_df.loc[idx,'text'] += face_text

# verify and save
assert(train_df.loc[0, 'text'][-10:] == "years old.")
train_df.to_csv('cyw_features/train_w_face.csv')

In [185]:
# Integrate Entities Labels

# include fairface labels?
include_face = True
if include_face:
    load_path = 'cyw_features/train_w_face.csv'
    save_path = 'cyw_features/train_w_face_n_entities.csv'
else:
    load_path = 'train.csv'
    save_path = 'cyw_features/train_w_entities'

# load files
train_df = pd.read_csv(load_path)
entities_features = pd.read_csv('cyw_features/entities_all.csv')

# make sure all original text ends with ending punctuation: . ! ?
for index, row in train_df.iterrows():
    if row['text'][-1].isalpha():
        train_df.loc[index, 'text'] += "."

# cancatenate race features to text
score_cols = entities_features.columns[1:]
for index, row in entities_features.iterrows():
    id = row['filename'][:5]

    entities_text = ''
    for i, c in enumerate(score_cols):
        if i % 2 == 1 and type(row[c]) == str: # valid description col
            entities_text += " "
            entities_text += row[c]

    idx = train_df[train_df['id'] == int(id)].index
    train_df.loc[idx,'text'] += entities_text

# verify and save
assert(train_df.loc[0, 'text'][-50:] == "Facial hair Forehead Hair Meter Face Pressure head")

train_df.to_csv(save_path)